[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ivanvladimir/maquinas_notebooks/blob/main/lfya/03%20Las%20m%C3%A1quina%20que%20est%C3%A1n%20en%20varios%20lugares.ipynb)

In [ ]:
# Execute if not installed
!pip install maquinas

In [ ]:
from maquinas.regular.dfa import DeterministicFiniteAutomaton as DFA
from maquinas.regular.ndfa import NonDeterministicFiniteAutomaton as NDFA
from maquinas.regular.ndfa_e import NonDeterministicFiniteAutomaton_epsilon as NDFA_e
from maquinas.regular.ndfa_e import epsilon
from maquinas.regular.reductions import *
from maquinas.simulation import Simulation

In [ ]:
abb=DFA(Q=['q_0','q_1','q_2','q_3'],
         sigma=['a','b'],
         q_0='q_0',
         A=['q_3'],
         delta=[
             (('q_0','a'),'q_1'),
             (('q_0','b'),'q_0'),
             (('q_1','a'),'q_1'),
             (('q_1','b'),'q_2'),
             (('q_2','a'),'q_1'),
             (('q_2','b'),'q_3'),
             (('q_3','a'),'q_3'),
             (('q_3','b'),'q_3'),
         ])
abb.save_img('contiene_abb')

In [ ]:
# Creating automata finito no deterministico
chicles_1=NDFA(Q=['q₀',
                  'q₁₁/₁','q₂₁/₁','q₃₁/₁','q₄₁/₁',
                  'q₁₁/₂','q₁₁_₁₂/₂',
                  'q₁₁/₃','q₁₁_₁₂/₃','q₂₁_₁₂/₃',
                  'q₁₁/₄','q₂₁/₄','q₂₁_₁₂/₄',
                  'q₁₁/₅','q₂₁/₅','q₃₁/₅',
                  'q₁₂/₆','q₂₂/₆',
                  'q₁₂/₇','q₁₁_₁₂/₇',
                  'q₁₂/₈','q₁₁_₁₂/₈','q₂₁_₁₂/₈',
                  'q₅'
                 ],
                         sigma=['1','2','5'],
                         q_0='q₀',
                         A=['q₅'],
                         delta=[
                            (('q₀','1'),['q₁₁/₁','q₁₁/₂','q₁₁/₃','q₁₁/₄','q₁₁/₅']),
                            (('q₀','2'),['q₁₂/₆','q₁₂/₇','q₁₂/₈']),
                            (('q₀','5'),['q₅']),
                            (('q₁₁/₁','1'),['q₂₁/₁']),
                            (('q₂₁/₁','1'),['q₃₁/₁']),
                            (('q₃₁/₁','1'),['q₄₁/₁']),
                            (('q₄₁/₁','1'),['q₅']),
                            (('q₁₁/₂','2'),['q₁₁_₁₂/₂']),
                            (('q₁₁_₁₂/₂','2'),['q₅']),
                            (('q₁₁/₃','2'),['q₁₁_₁₂/₃']),
                            (('q₁₁_₁₂/₃','1'),['q₂₁_₁₂/₃']),
                            (('q₂₁_₁₂/₃','1'),['q₅']),
                            (('q₁₁/₄','1'),['q₂₁/₄']),
                            (('q₂₁/₄','2'),['q₂₁_₁₂/₄']),
                            (('q₂₁_₁₂/₄','1'),['q₅']),
                            (('q₁₁/₅','1'),['q₂₁/₅']),
                            (('q₂₁/₅','1'),['q₃₁/₅']),
                            (('q₃₁/₅','2'),['q₅']),
                            (('q₁₂/₆','2'),['q₂₂/₆']),
                            (('q₂₂/₆','1'),['q₅']),
                            (('q₁₂/₇','1'),['q₁₁_₁₂/₇']),
                            (('q₁₁_₁₂/₇','2'),['q₅']),
                            (('q₁₂/₈','1'),['q₁₁_₁₂/₈']),
                            (('q₁₁_₁₂/₈','1'),['q₂₁_₁₂/₈']),
                            (('q₂₁_₁₂/₈','1'),['q₅']),                          
                         ])

In [ ]:
chicles_1.save_img("full")

In [ ]:
print(chicles_1.summary())

In [ ]:
chicles_1.table()

In [ ]:
for q,a,w_ in chicles_1.delta_stepwise("122"):
    if a:
        print(f"{a} -> {q}", end=",\n ")
    else:
        print(f"{q}",end="\n ")
    res=q
    
print(f"\nCon {res} Se acepta?", "Sí" if chicles_1.accept(res) else "No" )

In [ ]:
chicles_1.delta_extended(None,"122")

In [ ]:
s=Simulation(chicles_1,"122")
s.run()

In [ ]:
chicles_1.save_gif("122",'full.gif')

In [ ]:
chicles_2_full=ndfa2dfa(chicles_1,rename=False)

In [ ]:
chicles_2_full.table()

In [ ]:
chicles_2_full.save_img("dfa_full")

In [ ]:
chicles_2=ndfa2dfa(chicles_1)
chicles_2.table()

In [ ]:
chicles_2.save_img("dfa_chicles")

In [ ]:
for q,a,w_ in chicles_2.delta_stepwise("122"):
    if a:
        print(f"{a} -> {q}", end=",\n ")
    else:
        print(f"{q}",end="\n ")
    res=q
    
print(f"\nCon {res} Se acepta?", "Sí" if chicles_2.accept(res) else "No" )

In [ ]:
chicles_2.save_gif("122",'dfa.gif')

In [ ]:
chicles_3=NDFA_e(Q=chicles_2.Q,
                 sigma=chicles_2.sigma,
                 q_0=chicles_2.q_0,
                 A=chicles_2.A,
                 delta=[(k,[qs]) for k,qs in chicles_2.items()])

In [ ]:
for q_f in chicles_3.A:
    chicles_3.add_transition(q_f,'ε',['q_0'])

In [ ]:
chicles_3.save_img("ndfa_e_multiple")

In [ ]:
for q,a,w_ in chicles_3.delta_stepwise("5212"):
    if a:
        print(f"{a} -> {q}", end=",\n ")
    else:
        print(f"{q}",end="\n ")
    res=q
    
print(f"\nCon {res} Se acepta?", "Sí" if chicles_2.accept(res) else "No" )

In [ ]:
chicles_3.delta_extended(None,"5212")

In [ ]:
s=Simulation(chicles_3,"5212")
s.run()

In [ ]:
chicles_3.save_gif("5212","ndfa_e_multiple.gif")

In [ ]:
chicles_3.table()

In [ ]:
chicles_4=ndfa_e2ndfa(chicles_3, rename=False)
chicles_4.table()

In [ ]:
chicles_4.save_img("ndfa_multiple")

In [ ]:
chicles_4.save_gif("5212","ndfa_multiple.gif")

In [ ]:
chicles_5=ndfa2dfa(chicles_4)
chicles_5.save_img("dfa_multiple")

In [ ]:
chicles_5.save_gif("5212","dfa_multiple.gif")

In [ ]:
# agregando aceptar el pago: 2,2,2,2,2
chicles_5.add_transition('q_8','2','q_1')
chicles_5.graph()

In [ ]:
chicles_5.save_gif("22222","dfa_multiple_22222.gif")

In [ ]:
# Creating automata finito
ceros_uno=DFA(Q=['q_0','q_1'],
                         sigma=['0','1'],
                         q_0='q_0',
                         A=['q_1'],
                         delta=[
                            (('q_0','1'),'q_0'),
                            (('q_0','0'),'q_1')
                         ])
ceros_uno=dfa2ndfa_e(ceros_uno)
ceros_uno.save_img('unos_cero')

In [ ]:
uno_ceros=DFA(Q=['q_0','q_1'],
                         sigma=['0','1'],
                         q_0='q_0',
                         A=['q_1'],
                         delta=[
                            (('q_0','1'),'q_1'),
                            (('q_1','0'),'q_1')
                         ])
uno_ceros=dfa2ndfa_e(uno_ceros)
uno_ceros.save_img("uno_ceros")

In [ ]:
union=ceros_uno.union(uno_ceros)
union.save_img('union')

In [ ]:
concat=ceros_uno.concat(uno_ceros)
concat.save_img('concat')

In [ ]:
kleene=ceros_uno.kleene()
kleene.save_img("cerradura")

In [ ]:
from maquinas.regular.RE import RE
reg = RE('(a*ba*ba*)*+a*')
reg=reg.ndfa_e()

In [ ]:
reg.save_img("reg")

In [ ]:
reg_a = RE('a')
reg_a=reg_a.ndfa_e()
reg_a.save_img("reg_a")

In [ ]:
reg_a = RE('a*')
reg_a=reg_a.ndfa_e()
reg_a.save_img("reg_a_klene")

In [ ]:
reg_a_b = RE('a*b')
reg_a_b=reg_a_b.ndfa_e()
reg_a_b.save_img("reg_a_b_klene")